In [ ]:
!pip install biopython

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.8 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 16.3 MB/s eta 0:00:00


In [63]:
from io import StringIO
from Bio import Entrez, SeqIO
import os
# Set your NCBI email (Required for API usage)
Entrez.email = "petershamoun80@gmail.com"

# Define constants
MAX_SEQ_LENGTH = 10_000  # Maximum allowed sequence length
MAX_FETCH_ATTEMPTS = 5  # Maximum number of fetch attempts before giving up

# Define search queries
queries = {
    "protein_coding": '("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND ("CDS"[Feature] OR "mRNA"[Feature])',
    "non_protein_coding": '("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND ("ncRNA"[Feature] OR "lncRNA"[Feature])',
    "enhancer": '("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND ("enhancer"[Title] OR "enhancer"[Gene Name])',
    "non_enhancer": '("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND NOT ("enhancer"[Title] OR "enhancer"[Gene Name])',
    "promoter": '("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND "promoter"[Title]',
    "non_promoter": '("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND NOT "promoter"[Title]',
    "splice_site": '("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND ("splice site"[Title] OR "splice junction"[Title])',
    "non_splice_site": '("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND NOT ("splice site"[Title] OR "splice junction"[Title])',
    "methylated": '("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND ("methylation"[Title] OR "CpG island"[Title])',
    "non_methylated": '("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND NOT ("methylation"[Title] OR "CpG island"[Title])'
}

# Create 'sequences' directory if it doesn't exist
output_dir = "sequences"
os.makedirs(output_dir, exist_ok=True)

# Function to fetch sequences from NCBI and ensure we get 'max_records' valid sequences
def fetch_fasta(query, filename, max_records=10):
    print(f"Searching NCBI for: {query}")
    
    valid_sequences = []
    fetched_ids = set()
    attempt = 0

    while len(valid_sequences) < max_records and attempt < MAX_FETCH_ATTEMPTS:
        attempt += 1
        remaining = max_records - len(valid_sequences)
        
        # Step 1: Search NCBI to get sequence IDs
        handle = Entrez.esearch(db="nucleotide", term=query, retmax=remaining)
        record = Entrez.read(handle)
        handle.close()

        # Get new sequence IDs, avoiding duplicates
        new_ids = [seq_id for seq_id in record["IdList"] if seq_id not in fetched_ids]
        fetched_ids.update(new_ids)

        if not new_ids:
            print(f"No new sequences found in attempt {attempt}. Retrying...")
            continue

        print(f"Fetching {len(new_ids)} sequences...")

        # Step 2: Fetch FASTA sequences
        handle = Entrez.efetch(db="nucleotide", id=new_ids, rettype="fasta", retmode="text")
        fasta_records = handle.read()
        handle.close()

        # Step 3: Filter sequences by length
        fasta_io = StringIO(fasta_records)
        for seq_record in SeqIO.parse(fasta_io, "fasta"):
            if len(seq_record.seq) <= MAX_SEQ_LENGTH:
                valid_sequences.append(seq_record)
                if len(valid_sequences) == max_records:
                    break  # Stop early if we reach the target count

        print(f"Valid sequences collected: {len(valid_sequences)} (Target: {max_records})")

    # Step 4: Save only the filtered sequences to file
    if valid_sequences:
        file_path = os.path.join(output_dir, filename)
        with open(file_path, "w") as f:
            SeqIO.write(valid_sequences, f, "fasta")
        print(f"Saved {len(valid_sequences)} sequences to {file_path}.")
    else:
        print(f"No sequences within allowed length ({MAX_SEQ_LENGTH}) for {filename}.")

# Run the function for each query
for label, query in queries.items():
    fetch_fasta(query, f"{label}.fasta")

Searching NCBI for: ("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND ("CDS"[Feature] OR "mRNA"[Feature])
Fetching 10 sequences...
Valid sequences collected: 9 (Target: 10)
No new sequences found in attempt 2. Retrying...
No new sequences found in attempt 3. Retrying...
No new sequences found in attempt 4. Retrying...
No new sequences found in attempt 5. Retrying...
Saved 9 sequences to sequences\protein_coding.fasta.
Searching NCBI for: ("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND ("ncRNA"[Feature] OR "lncRNA"[Feature])
Fetching 10 sequences...
Valid sequences collected: 3 (Target: 10)
No new sequences found in attempt 2. Retrying...
No new sequences found in attempt 3. Retrying...
No new sequences found in attempt 4. Retrying...
No new sequences found in attempt 5. Retrying...
Saved 3 sequences to sequences\non_protein_coding.fasta.
Searching NCBI for: ("Homo sapiens"[Organism] OR "Mus musculus"[Organism]) AND ("enhancer"[Title] OR "enhancer"[Gene Name])
Fetchi

In [2]:
import os
import numpy as np

In [64]:


fasta_dir = "sequences"

# Mapping of base categories to questions
questions = {
    "protein_coding": "Does this nucleotide sequence encode a protein? Only answer Yes or No",
    "enhancer": "Does this nucleotide sequence function as an enhancer in gene regulation? Only answer Yes or No",
    "promoter": "Does this nucleotide sequence act as a promoter for transcription initiation? Only answer Yes or No",
    "splice_site": "Does this nucleotide sequence contain a splice site for RNA processing? Only answer Yes or No",
    "methylated": "Is this nucleotide sequence methylated as part of epigenetic regulation? Only answer Yes or No"
}

# Dictionary to hold sequence data
sequence_arrays = {}

# Loop through all FASTA files in the directory
for file in os.listdir(fasta_dir):
    if file.endswith(".fasta"):  # Process only FASTA files
        file_path = os.path.join(fasta_dir, file)
        var_name = os.path.splitext(file)[0]  # Use file name (without extension) as key

        # Determine base category
        if var_name.startswith("non_"):
            base_category = var_name[4:]  # Remove "non_" prefix
        else:
            base_category = var_name
        question = questions[base_category]  # Get the corresponding question

        # Read the FASTA file and create a list of dictionaries
        gene_dicts = []
        for record in SeqIO.parse(file_path, "fasta"):
            gene_dict = {
                "file_name": var_name,
                "gene": str(record.seq),
                "question": question,  # Add the question key
                "hallucination": "",
                "hallucination_classification": "",
                "classification": ""
            }
            gene_dicts.append(gene_dict)
        
        # Store the list of dictionaries in the dictionary
        sequence_arrays[var_name] = gene_dicts

        print(f"Stored {var_name} with {len(gene_dicts)} genes.")

Stored enhancer with 10 genes.
Stored methylated with 10 genes.
Stored non_enhancer with 10 genes.
Stored non_methylated with 10 genes.
Stored non_promoter with 10 genes.
Stored non_protein_coding with 3 genes.
Stored non_splice_site with 10 genes.
Stored promoter with 10 genes.
Stored protein_coding with 9 genes.
Stored splice_site with 10 genes.


In [65]:
from openai import OpenAI  #OpenAI API client
from dotenv import load_dotenv  #For loading environment variables from .env file

In [66]:
#Load API key from environment variables
load_dotenv()  #This loads variables from a .env file into environment variables
api_key = os.getenv("OPENAI_API_KEY")  #Get the OpenAI API key from environment variables
client = OpenAI(api_key=api_key)  #Initialize the OpenAI client with the API key

In [67]:
def generate_hallucination(gene_dict):
    response = client.chat.completions.create(
            model= 'gpt-4o-mini',  #Use the model specified in the constructor

            messages=
            [
            {"role": "system", "content": "You are an expert in genomics."},
            {"role": "user", "content": f"{gene_dict['gene']} Describe the gene in natural language:"}
        ],  #The conversation context to send to the API


            max_tokens=4096,  #Maximum length of the response

            temperature=0.7  #Controls randomness/creativity of the response
        )
    return response.choices[0].message.content

In [68]:
def generate_classification(gene_dict):
    response = client.chat.completions.create(
            model= 'gpt-4o-mini',  #Use the model specified in the constructor

            messages=
            [
            {"role": "system", "content": "You are an expert in genomics."},
            {"role": "user", "content": f"{gene_dict['gene']} {gene_dict['question']}"}
        ],  #The conversation context to send to the API


            max_tokens=4096,  #Maximum length of the response

            temperature=0.7  #Controls randomness/creativity of the response
        )
    return response.choices[0].message.content

In [69]:
def generate_hallucination_classification(gene_dict):
    response = client.chat.completions.create(
            model= 'gpt-4o-mini',  #Use the model specified in the constructor

            messages=
            [
            {"role": "system", "content": "You are an expert in genomics."},
            {"role": "user", "content": f"{gene_dict['gene']} {gene_dict['hallucination']} {gene_dict['question']}"}
        ],  #The conversation context to send to the API


            max_tokens=4096,  #Maximum length of the response

            temperature=0.7  #Controls randomness/creativity of the response
        )
    return response.choices[0].message.content

In [70]:
for file in sequence_arrays.keys():
    for gene in sequence_arrays[file]:
        gene['hallucination'] = generate_hallucination(gene)
        gene['hallucination_classification'] = generate_hallucination_classification(gene)
        gene['classification'] = generate_classification(gene)
        print(gene)

{'file_name': 'enhancer', 'gene': 'AGAGCGCGTTGTTAGAGCTGTTCCGAGCGGGCGTTCGGCTGTTGGCTGCCTTCTTCCCCCTCTTACACACTCGTGTATTATTTTGAGGTGGTGTTCGCAGAGTTTGAAAGAGAATTTTAAAAAGCCGCAAGCGTTTCGCTCTTTTATTTTTATAATCCCCTTCAATTTGGGGCTAAAAAAACAGGAAGGAAGAAAAGGAAATGAAATGTGGTAAAAGAAGCTAAAAGGTGCCTTTTAAAAGATCGTTGCTGTGAAGTGAAAAAAAATCTCCAGAGAACCCCCAAGCGCCGCGAGACCTCTTCCGAACCAAAGGAGTTTGCGCTCGCTCTCAGGGAAGAAGCGACCATTCATTCCGAGGAATAGCAGCCAATTAAAAGACAAATAAAAAAAGTTTGGAGTGGGGCGCGAGCAGGCGGGCGGCGCTCGGCGGGCGCGGGGCTCGGCGGGCGCAGGACGGACGGTGCCCGCCGCCCTCAGTGACGCCCGCGGGGAATGCGGAGCCGGCCGCCCGCGCCCAGCCGCCGCGCCTGCCTGCCGCCCGCGTCACGCGAGACCCGCAGGCCCGGGACGCCGCCGCGGGGCCCTCGAAGCGGATGAGCCGCGCGCCTCTGCCTGGCGCAGCCAACTAAAGCGGCGTGGATGATTCGCGACCTGAGCAAGATGTACCCGCAGACGCGCCACCCGGCACCGCATCAGCCTGCTCAACCCTTCAAATTTACAATTTCAGAATCCTGTGATCGGATTAAGGAAGAGTTTCAGTTTTTACAGGCTCAATACCACAGTCTGAAGCTGGAATGTGAGAAGCTCGCCAGCGAGAAGACAGAGATGCAGCGGCATTATGTCATGTATTATGAAATGTCCTATGGGTTGAACATAGAAATGCACAAGCAGGCAGAGATTGTTAAACGACTAAATGCTATCTGTGCACAGGTCATTCCTTTCCTGTCCCAAGAGCACCAGCAACA

In [73]:
import pandas as pd
all_genes = []
for file_name in sequence_arrays.keys():
    all_genes.extend(sequence_arrays[file_name])

df = pd.DataFrame(all_genes)
df['correct'] = df['file_name'].str.contains("non", case=True, na=False).map({True: "No", False: "Yes"})
df['hallucination_classification'] = df['hallucination_classification'].str.replace(".", "", regex=False)
df['classification'] = df['classification'].str.replace(".", "", regex=False)
df.head()

,file_name,gene,question,hallucination,hallucination_classification,classification,correct
0,enhancer,AGAGCGCGTTGTTAGAGCTGTTCCGAGCGGGCGTTCGGCTGTTGGC...,Does this nucleotide sequence function as an e...,The provided sequence appears to be a long nuc...,No,No,Yes
1,enhancer,AGAGCGCGTTGTTAGAGCTGTTCCGAGCGGGCGTTCGGCTGTTGGC...,Does this nucleotide sequence function as an e...,The provided sequence appears to be a long DNA...,No,No,Yes
2,enhancer,ACAGGCCGGACTCGCTCTGCTAGCTGACGGCCGCTCGACGCCATGG...,Does this nucleotide sequence function as an e...,The sequence you provided is a long stretch of...,Yes,No,Yes
3,enhancer,AGAGCGCGTTGTTAGAGCTGTTCCGAGCGGGCGTTCGGCTGTTGGC...,Does this nucleotide sequence function as an e...,The provided sequence appears to be a long DNA...,No,No,Yes
4,enhancer,AGCCTCGGCTCGTGGGTGCCGGAAGTGGAGGCGGTTGGTGGGGTTG...,Does this nucleotide sequence function as an e...,The provided sequence is a long DNA fragment t...,No,No,Yes


In [74]:
df['hallucination_correct'] = df['hallucination_classification'] == df['correct']
df['no_hallucination_correct'] = df['classification'] == df['correct']
df.head()

,file_name,gene,question,hallucination,hallucination_classification,classification,correct,hallucination_correct,no_hallucination_correct
0,enhancer,AGAGCGCGTTGTTAGAGCTGTTCCGAGCGGGCGTTCGGCTGTTGGC...,Does this nucleotide sequence function as an e...,The provided sequence appears to be a long nuc...,No,No,Yes,False,False
1,enhancer,AGAGCGCGTTGTTAGAGCTGTTCCGAGCGGGCGTTCGGCTGTTGGC...,Does this nucleotide sequence function as an e...,The provided sequence appears to be a long DNA...,No,No,Yes,False,False
2,enhancer,ACAGGCCGGACTCGCTCTGCTAGCTGACGGCCGCTCGACGCCATGG...,Does this nucleotide sequence function as an e...,The sequence you provided is a long stretch of...,Yes,No,Yes,True,False
3,enhancer,AGAGCGCGTTGTTAGAGCTGTTCCGAGCGGGCGTTCGGCTGTTGGC...,Does this nucleotide sequence function as an e...,The provided sequence appears to be a long DNA...,No,No,Yes,False,False
4,enhancer,AGCCTCGGCTCGTGGGTGCCGGAAGTGGAGGCGGTTGGTGGGGTTG...,Does this nucleotide sequence function as an e...,The provided sequence is a long DNA fragment t...,No,No,Yes,False,False


In [75]:
df['hallucination_correct'].mean(), df['no_hallucination_correct'].mean()

(np.float64(0.532608695652174), np.float64(0.4891304347826087))